In [1]:
import optuna
import numpy as np
import os
import pandas as pd
from tqdm.notebook import tqdm
import logging


# Убираем мусорный optuna-логгер
import warnings

os.chdir('../..')
warnings.filterwarnings("ignore", category=RuntimeWarning)
optuna.logging.set_verbosity(optuna.logging.WARNING)
logger = logging.getLogger("optuna")
logger.setLevel(logging.ERROR)


from scripts.NEURON_Sim_Wrapper import Network

# The Idea
Чтобы настроить параметры нейрона на биологически правдоподобные (возможно) попробуем стимулировать нейрон в соме и замерять его напряжение в пре-синаптических терминалах. Измерив пропорцию терминалов до которых успешно дошли импульсы (NetCon сработал), мы ожидаем:

### Одиночный имаульс:
95%-99% "удачных" терминалов - моторные

70%-95% "удачных" терминалов - обычные

### Частота:
...

In [2]:
# Вспомогательная функция
def compute_proportion(spike_times, soma_id):
    soma_spikes = len(spike_times[soma_id])
    terminal_ids = [k for k in spike_times.keys() if k != soma_id]
    terminal_spikes_counts = [len(spike_times[k]) for k in terminal_ids]
    mean_terminal_spikes = np.mean(terminal_spikes_counts)
    proportion = mean_terminal_spikes / soma_spikes if soma_spikes > 0 else 0
    return proportion


# ===================================
# Optuna Objective (neuron_id параметр)
# ===================================
def make_objective(neuron_id, pbar, best_tracker, verbose):
    def objective(trial):
        # ---------
        # Гиперпараметры
        # ---------
        dend_gnabar = trial.suggest_float("dend_gnabar_hh", 0.01, 0.20, log=True)
        dend_gkbar  = trial.suggest_float("dend_gkbar_hh", 0.005, 0.15, log=True)
        dend_gl     = trial.suggest_float("dend_gl_hh", 0.00005, 0.005, log=True)
        # ---------------------------
        # 1) Создаём модель
        # ---------------------------
        neuron_ids = [neuron_id]
        net = Network(neuron_ids)
        net.load_graphs()
        soma_params = {
            'L': 20,
            'diam': 20,
            'Ra': 100,
            'cm': 1,
            'gnabar_hh': 0.12,
            'gkbar_hh': 0.036,
            'gl_hh': 0.0003,
            'el_hh': -54.3
        }
        dend_params = {
            'L': 5.0,
            'diam': 1.0,
            'Ra': 100.0,
            'cm': 1.0,
            'gnabar_hh': dend_gnabar,
            'gkbar_hh': dend_gkbar,
            'gl_hh': dend_gl,
            'el_hh': -70.0
        }
        net.build_sections(
            soma_mechanism='hh',
            soma_params=soma_params,
            dendrite_mechanism='hh',
            dendrite_params=dend_params
        )
        net.connect_morphology()
        # ---------------------------
        # 2) Синапсы
        # ---------------------------
        syn_params = {'tau1': 0.5, 'tau2': 2.0, 'e': 0.0}
        netcon_params = {'threshold': 0.0, 'weight': 0.5, 'delay': 3.0}
        net.build_synapses(synapse_params=syn_params, netcon_params=netcon_params)
        # ---------------------------
        # 3) Рекорды
        # ---------------------------
        synapses_df = pd.read_csv("./Datasets/Generated/Syn_Conns(manual).csv")
        filtered_df = synapses_df[
            synapses_df['pre_neuron_id'].astype(str) == neuron_id
        ]
        pre_syn_seg_ids = list(set(filtered_df['pre_node_id'].astype(str)))
        soma_seg_id = [
            seg for (n, seg), val in net.sections.items() if val is net.somas[neuron_id]
        ][0]
        rec_sections = pre_syn_seg_ids + [soma_seg_id]
        net.setup_recording(sections=rec_sections)
        net.setup_stimulus(neurons=[neuron_id], start_time=10, duration=30, amplitude=0.5)
        # ---------------------------
        # 4) Симуляция
        # ---------------------------
        t, voltages = net.run(duration=100)
        spike_times, spike_amps = net.analyze(t, voltages)
        net.reset()
        # ---------------------------
        # 5) Оценка
        # ---------------------------
        proportion = compute_proportion(spike_times, soma_seg_id)
        # Ограничение сверху
        if proportion > 0.70:
            loss = 10.0 + proportion
        else:
            loss = abs(0.70 - proportion)
        # ---------
        # Обновляем лучший результат
        # ---------
        if loss < best_tracker["loss"]:
            best_tracker["loss"] = loss
            best_tracker["proportion"] = proportion
        # ---------
        # Обновляем tqdm
        # ---------
        if verbose:
            pbar.set_description(
                f"best_loss={best_tracker['loss']:.3f} | best_prop={best_tracker['proportion']:.3f}"
            )
            pbar.update(1)
        return loss
    return objective


# ================================
# Запуск Optuna с tqdm
# ================================
def run_optimization(neuron_id, n_trials=100, verbose=True):
    if verbose:
        print(f"Запуск оптимизации для нейрона {neuron_id} ({n_trials} trials)\n")
    best_tracker = {"loss": float("inf"), "proportion": 0.0}
    with tqdm(total=n_trials, disable=not verbose) as pbar:
        study = optuna.create_study(direction="minimize")
        study.optimize(
            make_objective(neuron_id, pbar, best_tracker, verbose),
            n_trials=n_trials,
            show_progress_bar=False
        )
    if verbose:
        print("\n==================== Лучшие параметры: ====================")
        print(study.best_params)
        print(f"Лучший loss: {study.best_value}")
        print(f"Лучший proportion: {best_tracker['proportion']}")
    return study, best_tracker

# Test on a single neuron

In [3]:
study = run_optimization(neuron_id="19431430", n_trials=10)
study[0].best_params

Запуск оптимизации для нейрона 19431430 (10 trials)



  0%|          | 0/10 [00:00<?, ?it/s]


==================== Лучшие параметры: ====================
{'dend_gnabar_hh': 0.010575317413105783, 'dend_gkbar_hh': 0.014570604046485977, 'dend_gl_hh': 0.000281297144004346}
Лучший loss: 0.1210526315789473
Лучший proportion: 0.5789473684210527


{'dend_gnabar_hh': 0.010575317413105783,
 'dend_gkbar_hh': 0.014570604046485977,
 'dend_gl_hh': 0.000281297144004346}

# Optimize 3k neurons

In [4]:
# Get only neurons with pre synaptic terminals available 

synapses_df = pd.read_csv("./Datasets/Generated/Syn_Conns(manual).csv")
neuron_ids = list(set((map(str, synapses_df["pre_neuron_id"].unique()))))[:]

len(neuron_ids)

3830

In [7]:
from multiprocessing import Pool
import traceback
import json
from datetime import datetime




# Обёртка чтобы multiprocessing мог вернуть результат
def optimize_single_neuron(neuron_id):
    try:
        study, best_tracker = run_optimization(neuron_id=neuron_id, n_trials=1, verbose=False)
        
        # print(f"Neuron: {neuron_id} \tBest Loss: {best_tracker['loss']:.3f}\tBest Prop: {best_tracker['proportion']:.3f}")
        
        res = {
            "best_params": study.best_params,
            "best_loss": best_tracker['loss'],
            "best_proportion": best_tracker['proportion']
        }
        # Сохраняем результаты в отдельный JSON файл
        save_path = os.path.join(save_dir, f"{neuron_id}.json")
        with open(save_path, "w", encoding="utf-8") as f:
            json.dump(res, f, indent=4, ensure_ascii=False)
        return neuron_id, res
    except Exception as e:
        traceback.print_exc()
        return neuron_id, {"error": str(e)}

    
    
    
    
    
n_processes = 72
results = {}
with Pool(n_processes) as pool:
    for neuron_id, res in tqdm(pool.imap_unordered(optimize_single_neuron, neuron_ids), total=len(neuron_ids)):
        results[neuron_id] = res

        
        
        
# Сохранение в общий JSON (если нужно)
save_path = "neurons_optimized_params(70).json"
with open(save_path, "w", encoding="utf-8") as f:
    json.dump(
        {
            "timestamp": datetime.now().isoformat(),
            "results": results
        },
        f,
        indent=4,
        ensure_ascii=False
    )
print(f"\nФайл успешно сохранён: {save_path}")

  0%|          | 0/3830 [00:00<?, ?it/s]


Файл успешно сохранён: neurons_optimized_params(70).json
